# Tool Use Chain of Thought (CoT) Enhancement

This notebook demonstrates how to prepare and enhance tool-use conversations with Chain of Thought (CoT) reasoning using Synthetic Data Kit.

## What We'll Do:

1. Load a tool-use dataset from Hugging Face
2. Format the data to include proper tool tags (`<tool></tool>`)
3. Create a small sample for demonstration
4. Generate a configuration file for CoT enhancement
5. Use the `cot-enhance` feature to add detailed reasoning to tool calls
6. Compare original vs. enhanced conversations

## Why CoT Enhancement Matters:

Chain of Thought enhancement makes the model's reasoning explicit before making tool calls. This helps:
- Make the model's decision process more transparent
- Provide better training examples for fine-tuning
- Improve model performance on complex reasoning tasks
- Create datasets that encourage step-by-step thinking

Let's get started!

In [1]:
from datasets import load_from_disk, load_dataset

In [2]:
dataset = load_dataset("Team-ACE/ToolACE")

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['system', 'conversations'],
        num_rows: 11300
    })
})

In [4]:
dataset['train']

Dataset({
    features: ['system', 'conversations'],
    num_rows: 11300
})

In [5]:
dataset['train'][0]

{'system': 'You are an expert in composing functions. You are given a question and a set of possible functions. \nBased on the question, you will need to make one or more function/tool calls to achieve the purpose. \nIf none of the function can be used, point it out. If the given question lacks the parameters required by the function,\nalso point it out. You should only return the function call in tools call sections.\nHere is a list of functions in JSON format that you can invoke:\n[{"name": "newAddress", "description": "Generates a new Ethereum address that can be used to send or receive funds. Do not lose the password! We can\'t restore access to an address if you lose it.", "parameters": {"type": "dict", "properties": {"password": {"description": "The password for the new Ethereum address", "type": "string"}}, "required": ["password"]}, "required": null}, {"name": "Market Trends API", "description": "Get the latest market trends and relevant news for a specified country and languag

In [7]:
cot = load_from_disk("./CoT_ToolACE/")

In [11]:
cot['train'][0]

{'id': '5c1710db-5977-4fb4-b5e2-edc83c34d761',
 'conversations': [{'from': 'system',
   'value': 'You are ToolLlama, whenever asked a question, requiring tool call you responed after thinking and returning tool call within <tool></tool> XML tags.\nYou are an expert in composing functions. You are given a question and a set of possible functions. \nBased on the question, you will need to make one or more function/tool calls to achieve the purpose. \nIf none of the function can be used, point it out. If the given question lacks the parameters required by the function,\nalso point it out. .\nHere is a list of functions in JSON format that you can invoke:\n[{"name": "newAddress", "description": "Generates a new Ethereum address that can be used to send or receive funds. Do not lose the password! We can\'t restore access to an address if you lose it.", "parameters": {"type": "dict", "properties": {"password": {"description": "The password for the new Ethereum address", "type": "string"}}, "

In [21]:
from datasets import load_dataset
import multiprocessing
import uuid

# Load the ToolACE dataset
dataset = load_dataset("Team-ACE/ToolACE")

# Define the transformation function for batched processing
def transform_examples(examples, indices):
    tool_llama_prefix = "You are ToolLlama, whenever asked a question, requiring tool call you responed after thinking and returning tool call within <tool></tool> XML tags.\n"
    
    # Initialize the output lists
    ids = []
    new_conversations_list = []
    
    # Process each example in the batch
    for i in range(len(indices)):
        # Generate a unique ID
        unique_id = str(uuid.uuid4())
        ids.append(unique_id)
        
        # Extract and modify system message content
        if isinstance(examples['system'][i], str):
            system_content = tool_llama_prefix + examples['system'][i]
        elif isinstance(examples['system'][i], dict) and 'value' in examples['system'][i]:
            system_content = tool_llama_prefix + examples['system'][i]['value']
        else:
            system_content = tool_llama_prefix + str(examples['system'][i])
        
        # Create new conversations array with system message as first item
        new_conversations = [
            {
                "from": "system",
                "value": system_content
            }
        ]
        
        # Add the rest of the conversation messages
        new_conversations.extend(examples['conversations'][i])
        
        # Wrap function calls with <tool></tool> tags
        for j in range(len(new_conversations)):
            # Check if current message is from assistant
            if new_conversations[j]["from"] == "assistant":
                assistant_value = new_conversations[j]["value"]
                
                # Case 1: Assistant message followed by tool response
                if (j+1 < len(new_conversations) and new_conversations[j+1]["from"] == "tool"):
                    if assistant_value.strip().startswith('[') and assistant_value.strip().endswith(']'):
                        new_conversations[j]["value"] = f"<tool>{assistant_value}</tool>"
                
                # Case 2: Assistant message is the last message
                elif j == len(new_conversations) - 1:
                    if assistant_value.strip().startswith('[') and assistant_value.strip().endswith(']'):
                        new_conversations[j]["value"] = f"<tool>{assistant_value}</tool>"
        
        new_conversations_list.append(new_conversations)
    
    # Return the batch of transformed examples
    return {
        "id": ids,
        "conversations": new_conversations_list
    }

# Process with parallelism
def process_dataset():
    # Use transform_examples for batched processing
    return dataset.map(
        function=transform_examples,
        with_indices=True,
        batched=True,
        batch_size=100,  # Process in larger batches
        num_proc=multiprocessing.cpu_count(),  # Use all available CPU cores
        desc="Transforming dataset",  # Add progress description
        remove_columns=["system"]  # Remove the old system column
    )

# Run the transformation
transformed_dataset = process_dataset()

# Save the transformed dataset
transformed_dataset.save_to_disk("toolllama_formatted_dataset")

# Optional: Print an example to verify the transformation
if len(transformed_dataset["train"]) > 0:
    example = transformed_dataset["train"][0]
    print(f"ID: {example['id']}")
    print(f"First message: {example['conversations'][0]}")

Transforming dataset (num_proc=16):   0%|          | 0/11300 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/11300 [00:00<?, ? examples/s]

ID: 0f3721b9-6082-4635-8170-71dd61ebedc9
First message: {'from': 'system', 'value': 'You are ToolLlama, whenever asked a question, requiring tool call you responed after thinking and returning tool call within <tool></tool> XML tags.\nYou are an expert in composing functions. You are given a question and a set of possible functions. \nBased on the question, you will need to make one or more function/tool calls to achieve the purpose. \nIf none of the function can be used, point it out. If the given question lacks the parameters required by the function,\nalso point it out. You should only return the function call in tools call sections.\nHere is a list of functions in JSON format that you can invoke:\n[{"name": "newAddress", "description": "Generates a new Ethereum address that can be used to send or receive funds. Do not lose the password! We can\'t restore access to an address if you lose it.", "parameters": {"type": "dict", "properties": {"password": {"description": "The password fo

In [22]:
test = load_from_disk("./toolllama_formatted_dataset/")

In [23]:
test['train']

Dataset({
    features: ['conversations', 'id'],
    num_rows: 11300
})

In [26]:
test['train'][0]

{'conversations': [{'from': 'system',
   'value': 'You are ToolLlama, whenever asked a question, requiring tool call you responed after thinking and returning tool call within <tool></tool> XML tags.\nYou are an expert in composing functions. You are given a question and a set of possible functions. \nBased on the question, you will need to make one or more function/tool calls to achieve the purpose. \nIf none of the function can be used, point it out. If the given question lacks the parameters required by the function,\nalso point it out. You should only return the function call in tools call sections.\nHere is a list of functions in JSON format that you can invoke:\n[{"name": "newAddress", "description": "Generates a new Ethereum address that can be used to send or receive funds. Do not lose the password! We can\'t restore access to an address if you lose it.", "parameters": {"type": "dict", "properties": {"password": {"description": "The password for the new Ethereum address", "type

In [ ]:
import json
import os
import random
from pathlib import Path

# Create a directory for saving the tool example files 
os.makedirs(\"../../tool_examples\", exist_ok=True)

# Number of examples to use in the sample
NUM_EXAMPLES = 10

# Get a sample of conversations from the dataset
sample_indices = random.sample(range(len(test['train'])), NUM_EXAMPLES)
sample_conversations = [test['train'][i] for i in sample_indices]

# Format for cot-enhance: Each conversation should be in the expected structure
formatted_conversations = []

for example in sample_conversations:
    # Each example is already in the right format with XML tags for tool calls
    formatted_conversations.append({
        \"conversations\": example[\"conversations\"]
    })

# Now save this as a JSON file in format that cot-enhance expects
# For multiple conversations, we'll create a list of conversation objects
output_file = \"../../tool_examples/multi_conversations.json\"
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(formatted_conversations, f, indent=2)

print(f\"Sample of {NUM_EXAMPLES} conversations saved to {output_file}\")

In [ ]:
# Now let's test the cot-enhance feature with our generated file
# First, we'll create the custom config file for cot-enhance

import yaml
import os

# Create the config directory if it doesn't exist
config_dir = os.path.dirname(os.path.abspath(\"cot_tools_config.yaml\"))
os.makedirs(config_dir, exist_ok=True)

# Custom config for CoT enhancement
cot_config = {
    \"vllm\": {
        \"api_base\": \"http://localhost:8000/v1\",
        \"model\": \"unsloth/Meta-Llama-3.1-8B-Instruct\",
        \"max_retries\": 3,
        \"retry_delay\": 1.0
    },
    \"generation\": {
        \"temperature\": 0.2,
        \"top_p\": 0.95,
        \"max_tokens\": 8192
    },
    \"prompts\": {
        \"cot_enhancement\": \"\"\"You are a high 170IQ reasoning super smart AI, your job is to enhance existing conversation examples. Remember return the entire conversation as is BUT

BUT We are add Chain of Thought and planning to "Assistant" messages whenever it returns a tool call.

Remember ONLY When it does return a tool, we all add thinking and reasoning Traces before it to add logic otherwise we don't touch the conversation history

Remember to return the entire message but only enhance the assistant messages whenever it calls a tool with thoghts

Please keep in mind we are not modifying anything in the example neither are we changing what it does, only add CoT everytime a tool gets called in the conversation

Think out loud and max out your tokens when adding CoT\"\"\"
    }
}

# Save the config file
with open(\"cot_tools_config.yaml\", 'w') as f:
    yaml.dump(cot_config, f, default_flow_style=False, sort_keys=False)

print(\"Custom cot-enhance config file saved to cot_tools_config.yaml\")

In [ ]:
# Now we'll run the cot-enhance command on our sample file
# This is how we would run it from the command line
# synthetic-data-kit -c cot_tools_config.yaml create ../../tool_examples/multi_conversations.json --type cot-enhance -o ../../enhanced_results/

# In this notebook, we'll test it by importing and calling the function directly
# This is just for demonstration purposes - in real usage, you'd use the CLI command

import os
from synthetic_data_kit.core.create import process_file
from pathlib import Path

# Create the output directory
os.makedirs(\"../../enhanced_results\", exist_ok=True)

# Call the function directly (similar to what the CLI does)
input_file = \"../../tool_examples/multi_conversations.json\"
output_dir = \"../../enhanced_results\"
config_path = Path(\"cot_tools_config.yaml\").absolute()
content_type = \"cot-enhance\"
verbose = True

# This simulates what happens when you run the CLI command
try:
    output_path = process_file(
        file_path=input_file,
        output_dir=output_dir,
        config_path=config_path,
        api_base=None,  # Will use config file setting
        model=None,     # Will use config file setting
        content_type=content_type,
        num_pairs=None,
        verbose=verbose
    )
    print(f\"Enhanced conversations saved to: {output_path}\")
    
    # Let's look at one example of the enhanced conversation
    # We'll load the enhanced file and compare it to the original
    import json
    
    with open(output_path, 'r') as f:
        enhanced_data = json.load(f)
    
    if isinstance(enhanced_data, list):
        # We have multiple conversations
        print(f\"Enhanced {len(enhanced_data)} conversations\")
        # Let's examine the first conversation
        print(\"\\nFirst Enhanced Conversation Sample:\")

In [ ]:
        # Let's examine a tool call from the first conversation
        for i, conv in enumerate(enhanced_data):
            conversations = conv.get(\"conversations\", [])
            for j, message in enumerate(conversations):
                if message.get(\"from\") == \"assistant\" and \"<tool>\" in message.get(\"value\", \"\"):
                    print(f\"\\nBefore and After Enhancement Example (Conversation {i+1}, Message {j+1}):\\n\")
                    
                    # Original version
                    print(\"ORIGINAL (from original dataset):\")
                    print(\"-\" * 80)
                    original_tool_call = sample_conversations[i][\"conversations\"][j][\"value\"]
                    print(original_tool_call)
                    print(\"-\" * 80)
                    
                    # Enhanced version
                    print(\"\\nENHANCED (with Chain of Thought):\")
                    print(\"-\" * 80)
                    enhanced_tool_call = message[\"value\"]
                    print(enhanced_tool_call)
                    print(\"-\" * 80)
                    
                    # Exit after first example
                    print(\"\\nNote how the enhanced version includes detailed reasoning steps before making the tool call.\\n\")
                    raise StopIteration
    
    elif isinstance(enhanced_data, dict) and \"conversations\" in enhanced_data:
        # We have a single conversation
        print(\"Enhanced a single conversation\")
        for j, message in enumerate(enhanced_data[\"conversations\"]):
            if message.get(\"from\") == \"assistant\" and \"<tool>\" in message.get(\"value\", \"\"):
                print(f\"\\nBefore and After Enhancement Example (Message {j+1}):\\n\")
                
                # Original version
                print(\"ORIGINAL (from original dataset):\")
                print(\"-\" * 80)
                original_tool_call = sample_conversations[0][\"conversations\"][j][\"value\"]
                print(original_tool_call)
                print(\"-\" * 80)
                
                # Enhanced version
                print(\"\\nENHANCED (with Chain of Thought):\")
                print(\"-\" * 80)
                enhanced_tool_call = message[\"value\"]
                print(enhanced_tool_call)
                print(\"-\" * 80)
                
                # Exit after first example
                print(\"\\nNote how the enhanced version includes detailed reasoning steps before making the tool call.\\n\")
                break
except StopIteration:
    # Used to break out of nested loops
    pass
except Exception as e:
    print(f\"Error: {e}\")